# <font color='008fd0'>Overview</font>

This notebooks represents the processing steps for the patient dataset.

## <font color='00b269'>Workflow</font>
- **SimpleTransform** - keep and rename only useful columns
- **TransformDates** - extract age for date and birthdate
- **TransformDiagnostic** - extract diagnostics and comorbidities
- **GroupByPatient** - aggregate rows from the same patient
- **DropNa** - drop nan rows
- **Pandas2Spark** - transform df to spark
- **AppendFilter** - concatenate all dfs
- **Count** - count all patients
- **JoinMeteo** - join with meteo data

In [1]:
import pandas as pd
import warnings
from tqdm import tqdm
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from pyspark.sql.functions import col, year

StatementMeta(, d3479b2c-6cd3-4d50-ab84-59a79f186f05, 3, Finished, Available)

In [2]:
folder = "abfss://Medio@onelake.dfs.fabric.microsoft.com/Medio.Lakehouse/Files/Mobile/Source"

StatementMeta(, d3479b2c-6cd3-4d50-ab84-59a79f186f05, 4, Finished, Available)

In [3]:
excel_1 = pd.ExcelFile(f"{folder}/CJ_SC_1.xlsx")
print(excel_1.sheet_names)
df_2008 = excel_1.parse('SC_2008')
df_2009 = excel_1.parse('SC_2009')
df_2010 = excel_1.parse('SC_2010')

StatementMeta(, d3479b2c-6cd3-4d50-ab84-59a79f186f05, 5, Finished, Available)

['SC_2008', 'SC_2009', 'SC_2010', 'Sheet2']


In [4]:
excel_2 = pd.ExcelFile(f"{folder}/CJ_SC_2.xlsx")
print(excel_2.sheet_names)
df_2011 = excel_2.parse('CJ_SC_2011')
df_2011 = df_2011[df_2011['DataNastere'] != "00/01/1900"]
df_2012 = excel_2.parse('CJ_SC_2012')
df_2013 = excel_2.parse('CJ_SC_2013')

StatementMeta(, d3479b2c-6cd3-4d50-ab84-59a79f186f05, 6, Finished, Available)

['CJ_SC_2011', 'CJ_SC_2012', 'CJ_SC_2013', 'Sheet1']


In [5]:
excel_3 = pd.ExcelFile(f"{folder}/CJ_SC_3.xlsx")
print(excel_3.sheet_names)
df_2007 = excel_3.parse('CJ_SC_2007')
df_2014 = excel_3.parse('CJ_SC_2014')
df_2015 = excel_3.parse('CJ_SC_2015')
df_2016 = excel_3.parse('CJ_SC_2016')

StatementMeta(, d3479b2c-6cd3-4d50-ab84-59a79f186f05, 7, Finished, Available)

['CJ_SC_2014', 'CJ_SC_2015', 'CJ_SC_2016', 'CJ_SC_2007']


In [6]:
columns = ['IdPacientFisa', 'Sex', 'DataNastere', 'CriteriuInternare', 'DataInternare', 'CodDiagn_princ', 'CodDiagn_sec']
rename_map = {
    'IdPacientFisa': 'ID',
    'DataNastere': 'BirthDate',
    'CriteriuInternare': 'Type',
    'DataInternare': 'Date',
    'CodDiagn_princ': 'Diagnostic',
    'CodDiagn_sec': 'Diseases'
}

StatementMeta(, 7224a8f6-9957-41ca-87a1-e562b5bd882f, 8, Finished, Available)

In [ ]:
dfs = [df_2007, df_2008, df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016]

In [14]:
class SimpleTransform(TransformerMixin):
    def __init__(self, columns, rename_map):
        self.columns = columns
        self.rename_map = rename_map
    def transform(self, origin):
        Xs = []
        for idx, df in enumerate(origin):
            X = df[columns]
            X.rename(columns=rename_map, inplace=True)
            X['BirthDate'] = pd.to_datetime(X['BirthDate'], errors='coerce').dt.date
            X.drop_duplicates(inplace=True)
            Xs.append(X)
        return Xs

StatementMeta(, 7224a8f6-9957-41ca-87a1-e562b5bd882f, 16, Finished, Available)

In [101]:
class TransformDates(TransformerMixin):
    def transform(self, Xs):
        for idx, X in enumerate(Xs):
            birth_dates = pd.to_datetime(X['BirthDate'], errors='coerce').dt.date
            X['Date'] = pd.to_datetime(X['Date'], errors='coerce').dt.date
            X['Age'] = ((X['Date'] - birth_dates) // 365).apply(lambda x: x.days)
            X.drop(columns=['BirthDate'], inplace=True)
            Xs[idx] = X
        return Xs

StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 103, Finished, Available)

In [102]:
class TransformDiagnostic(TransformerMixin):
    def transform(self, Xs):
        for idx, X in enumerate(Xs):
            X['Diagnostic'] = X['Diagnostic'].apply(lambda x: str(x)[:3])
            X['Diseases'] = X['Diseases'].apply(lambda x: str(x)[:3])

            features = {'I10': 'HT', 'I48': 'AF', 'I25':'CIHD', 'J44': 'COPD'}
            for code, feature in features.items():
                X[feature] = (X['Diseases'] == code).astype(int)

            multiple_codes = ['I34', 'I35', 'I36', 'I37']
            for code in multiple_codes:
                X['VI'] = (X['Diseases'].isin(multiple_codes)).astype(int)

            targets ={'I50': 'HF', 'J96': 'RF', 'I63': 'CI'}
            for code, target in targets.items():
                X[target] = (X['Diagnostic'] == code).astype(int)
            Xs[idx] = X
        return Xs

StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 104, Finished, Available)

In [111]:
class GroupByPatient(TransformerMixin):
    def transform(self, Xs):
        for idx, X in enumerate(Xs):
            features = ['HT', 'AF', 'CIHD', 'COPD', 'VI']
            for feature in features:
                X[feature] = X.groupby('ID')[feature].transform('max')

            X.drop(columns=['Diseases', 'Diagnostic'], inplace=True)
            X.drop_duplicates(inplace=True)
            Xs[idx] = X
        return Xs

StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 113, Finished, Available)

In [112]:
class DropNa(TransformerMixin):
    def transform(self, Xs):
        for idx, X in enumerate(Xs):
            X.dropna(axis=0, inplace=True)
            Xs[idx] = X
        return Xs

StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 114, Finished, Available)

In [113]:
class Pandas2Spark(TransformerMixin):
    def transform(self, Xs):
        for idx, X in enumerate(Xs):
            X = spark.createDataFrame(X)
            Xs[idx] = X
        return Xs

StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 115, Finished, Available)

In [123]:
class AppendFilter(BaseEstimator, TransformerMixin):
    def transform(self, Xs):
        dataset = Xs[0]
        for idx, X in enumerate(Xs[1:]):
            dataset = dataset.union(X)

        dataset = dataset.filter((year('Date') >= 2007) & (year('Date') <= 2016))
        return dataset

StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 125, Finished, Available)

In [124]:
class JoinMeteo(BaseEstimator, TransformerMixin):
    def transform(self, X):
        meteo = spark.sql("SELECT * FROM Meteo WHERE City = 'Cluj Napoca'")
        X = X.join(meteo, ['Date'])
        return X

StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 126, Finished, Available)

In [125]:
pipeline = Pipeline(steps=[
    ('simple', SimpleTransform(columns, rename_map)),
    ('dates', TransformDates()),
    ('codes', TransformDiagnostic()),
    ('groupby', GroupByPatient()),
    ('dropna', DropNa()),
    ('2Spark', Pandas2Spark()),
    ('append', AppendFilter()),
    ('meteo', JoinMeteo())
])
warnings.filterwarnings("ignore")
test = pipeline.transform(dfs)
warnings.filterwarnings("default")


StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 127, Finished, Available)

In [126]:
test.count()

StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 128, Finished, Available)

1566773

In [127]:
test.head(5)

StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 129, Finished, Available)

/home/trusted-service-user/cluster-env/trident_env/lib/python3.10/socket.py:776: ResourceWarning: unclosed <socket.socket fd=68, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('127.0.0.1', 52548), raddr=('127.0.0.1', 41119)>
  self._sock = None


[Row(Date=datetime.date(2007, 1, 22), ID=6682, Sex='M', Type='Tratament', Age=52, HT=1, AF=0, CIHD=1, COPD=0, VI=0, HF=0, RF=0, CI=0, Max=6.8, Min=-0.8, Prec=0.0, Press=1019.8, Wind=7, Insolat=1.9),
 Row(Date=datetime.date(2007, 1, 24), ID=6685, Sex='M', Type='Urgenta', Age=41, HT=0, AF=0, CIHD=0, COPD=0, VI=0, HF=0, RF=0, CI=0, Max=11.1, Min=5.4, Prec=2.7, Press=998.3, Wind=10, Insolat=1.0),
 Row(Date=datetime.date(2007, 1, 24), ID=6686, Sex='F', Type='Urgenta', Age=40, HT=0, AF=0, CIHD=0, COPD=0, VI=0, HF=0, RF=0, CI=0, Max=11.1, Min=5.4, Prec=2.7, Press=998.3, Wind=10, Insolat=1.0),
 Row(Date=datetime.date(2007, 1, 9), ID=6687, Sex='F', Type='Urgenta', Age=28, HT=0, AF=0, CIHD=0, COPD=0, VI=0, HF=0, RF=0, CI=0, Max=9.4, Min=1.9, Prec=0.0, Press=1022.2, Wind=4, Insolat=4.0),
 Row(Date=datetime.date(2007, 1, 8), ID=6689, Sex='F', Type='Urgenta', Age=73, HT=1, AF=0, CIHD=1, COPD=0, VI=0, HF=0, RF=0, CI=0, Max=7.2, Min=1.0, Prec=0.1, Press=1020.6, Wind=4, Insolat=0.4)]

In [120]:
test.write.mode("overwrite").parquet("Files/Mobile/DataSet")

StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 122, Finished, Available)

In [122]:
test.write.mode("overwrite").format("delta").save("Tables/Mobile")

StatementMeta(, 15482044-dc45-44bf-914e-4485468f10f7, 124, Finished, Available)